In [1]:
import pandas as pd
import numpy as np
import csv
import re

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [2]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [3]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        price: pos[0]: float
        descuento: pos[1]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").replace(",", ".").split("\n")
    
    # 4. Extracts and formats price value:
    price = float(x[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(x) == 1 else float(x[1])][0]
    
    return [price, discount]

In [7]:
pwd

'/Users/Moni/Ironhack/my_ironhack/projects/final-project/notebooks/transformations'

In [20]:
df = pd.read_csv("../../data/scrape-consum_2024-03-11.csv")
df

,categoria,subcategoria,descripcion,unidades,precio,marca
0,Despensa,Aperitivos y frutos secos,Aceitunas Negras sin Hueso Lata 150 Gr,"7,93 € / 1 Kg","1,29 €\n1,19 €",CONSUM
1,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Natural Tostado Bolsa 2...,"4,00 € / 1 Kg","1,05 €\n1,00 €",CONSUM
2,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Tostados con Sal Bolsa ...,"4,00 € / 1 Kg","1,05 €\n1,00 €",CONSUM
3,Despensa,Aperitivos y frutos secos,Ganchito Maíz Sabor Queso Bolsa 80 Gr,"8,75 € / 1 Kg","0,75 €\n0,70 €",CONSUM
4,Despensa,Aperitivos y frutos secos,Aceitunas Rellenas Anchoa Suave Pack 3 3 x 50 Gr,"17,66 € / 1 Kg","2,99 €\n2,65 €",JOLCA
...,...,...,...,...,...,...
11294,Mascotas,Comida para gatos,Comida Húmeda Gatos Gourmet Perle Buey Sobr 4 ...,"9,82 € / 1 Kg","3,34 €",GOURMET
11295,Mascotas,Comida para gatos,"Comida Seca Gatos Ultima de Pollo 1,5 Kg","6,63 € / 1 Kg","9,95 €",ULTIMA
11296,Mascotas,Comida para gatos,Snacks para Gatos Felix Party Mix Original 60 Gr,"30,83 € / 1 Kg","1,85 €",FELIX
11297,Mascotas,Comida para gatos,Comida Seca Gatos Esterilizados Buey 800 Gr,"7,44 € / 1 Kg","5,95 €",ULTIMA


## Duplicados

In [24]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 2312 duplicate rows


In [25]:
df.drop_duplicates(inplace=True)

In [26]:
# All collumns have values. 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8987 entries, 0 to 11298
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   categoria      8987 non-null   object 
 1   subcategoria   8987 non-null   object 
 2   descripcion    8987 non-null   object 
 3   unidades       8987 non-null   object 
 4   marca          8512 non-null   object 
 5   price_eu       8987 non-null   float64
 6   price_disc_eu  943 non-null    float64
dtypes: float64(2), object(5)
memory usage: 561.7+ KB


## PRECIO

### Precio

- No hay valores nulos

- El precio puede contener o 2 informaciones o 3:
    - El precio y a que hace referencia ese precio (pack)
    - El precio, precio si esta descontado y a que hace referencia ese precio(pack)

- Creamos una funcion que aplicamos a todos los elementos de la columna precio para formaterla. Aqui extraeremos los tres valores comentados antes. Cuando no haya descuento --> np.nan
    - price_info --> string
    - price_eu and price_disc_eu --> float
    
  **Note:** Interesante aqui el tema de apply(function) y luego el lambda. 
  
- Eliminamos la columna original

**NOTE:** En price_info, los valores son bastante consistentes siendo la mayoria de valoes (ud/pack). No obstante hay 14 valores que son de tipo ( xxx g) ej 500 g. Veremos a ver que hacemos. si los descartamos o como los tranformamos. No son una parte muy relevante del conjunto de datos ( 14 de 5035) pero tener en cuenta que hay que revisarlo: 


**Tranform Price column**

In [22]:
# Apply the function and get the element for each colum: 
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

df.sample()

,categoria,subcategoria,descripcion,unidades,marca,price_eu,price_disc_eu
10697,Cuidado personal,Maquillaje,Perfilador de Cejas Slim 06 1 U,"6,99 € / 1 U",MAYBELLINE,6.99,NaN


### UNIDADES:

In [117]:
def format_units(i):
    
    pattern_1 = r'(\d+\s*x\s*\d+(?:,\d+)?\s*[Gr|Kg|U|ml|L]+)$'
    
    if " x " in i:
        matches = re.findall(pattern_1, i)
        
        if matches != []:
            cantidad = matches[0]
        else:
            cantidad = np.nan
        return cantidad
            #Este va bastante bien:
            #print(f'{i} --- {cantidad}')
               
    else:
        units = ' '.join(i.split(" ")[-2:])
        
        if units.lower().replace(" ","").replace(".","").isalpha():
            
            cantidad = np.nan
        else:
            
            cantidad = f"1 x {units}"
        return cantidad

In [118]:
df["cantidad"] = df["descripcion"].apply(format_units)

## Formato

In [124]:
units = ['Pack', 'Unidades', 'Lata', 'Botella', 'Bolsa', 
                    'Frasco', 'Paquete', 'Brik', 'Tarro', 'Cápsulas', 
                    'Bote', 'Tarrina', 'Caja', 'Bandeja', 'Barra', 'Bolsitas', 
                    'Tarrito']

In [133]:
def format_format(x):
    
    units = ['Pack', 'Unidades', 'Lata', 'Botella', 'Bolsa', 'Frasco', 'Paquete', 'Brik', 
             'Tarro', 'Cápsulas', 'Bote', 'Tarrina', 'Caja', 'Bandeja', 'Barra', 'Bolsitas', 'Tarrito']
    
    for i in units:
        if i.lower() in x.lower():
            #format_ = units
            return i
        else:
            return np.nan

In [134]:
df["fromato"] = df["descripcion"].apply(format_format)

In [135]:
df

,categoria,subcategoria,descripcion,unidades,marca,price_eu,price_disc_eu,cantidad,fromato
0,Despensa,Aperitivos y frutos secos,Aceitunas Negras sin Hueso Lata 150 Gr,"7,93 € / 1 Kg",CONSUM,1.29,1.19,1 x 150 Gr,NaN
1,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Natural Tostado Bolsa 2...,"4,00 € / 1 Kg",CONSUM,1.05,1.00,1 x 250 Gr,NaN
2,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Tostados con Sal Bolsa ...,"4,00 € / 1 Kg",CONSUM,1.05,1.00,1 x 250 Gr,NaN
3,Despensa,Aperitivos y frutos secos,Ganchito Maíz Sabor Queso Bolsa 80 Gr,"8,75 € / 1 Kg",CONSUM,0.75,0.70,1 x 80 Gr,NaN
4,Despensa,Aperitivos y frutos secos,Aceitunas Rellenas Anchoa Suave Pack 3 3 x 50 Gr,"17,66 € / 1 Kg",JOLCA,2.99,2.65,3 x 50 Gr,Pack
...,...,...,...,...,...,...,...,...,...
11294,Mascotas,Comida para gatos,Comida Húmeda Gatos Gourmet Perle Buey Sobr 4 ...,"9,82 € / 1 Kg",GOURMET,3.34,NaN,4 x 85 Gr,NaN
11295,Mascotas,Comida para gatos,"Comida Seca Gatos Ultima de Pollo 1,5 Kg","6,63 € / 1 Kg",ULTIMA,9.95,NaN,"1 x 1,5 Kg",NaN
11296,Mascotas,Comida para gatos,Snacks para Gatos Felix Party Mix Original 60 Gr,"30,83 € / 1 Kg",FELIX,1.85,NaN,1 x 60 Gr,NaN
11297,Mascotas,Comida para gatos,Comida Seca Gatos Esterilizados Buey 800 Gr,"7,44 € / 1 Kg",ULTIMA,5.95,NaN,1 x 800 Gr,NaN


In [53]:
lista
re_words = ["Consum", 'Gr', 'de', 'U', 'ml', 'L', 'x', 'con', 'y', 'Queso', 
            'Chocolate', 'en', 'Crema', 'sin', 'Leche', 'Vino', 'Natural', 
            'Gel', 'Colonia', 'Aceite', 'Kg', 'Pollo', 'Extra', 'Tinte', 
            'Sabor', 'Mini', 'Café', 'Tinto', 'Blanco', 'Azúcar', 'Galleta', 
            'Refresco', 'Jamón', 'Salsa',  'Lv', 'Sin', 'Oliva', 'Champú', 
            'Bebida', 'Cerveza', 'Pan', 'Líquido', 'Fresa', 'Cacao', 
            'Lonchas', 'Desodorante', 'Limón', 'Yogur', 'Tomate', 
            'Negro', 'Sal', 'Cereales', 'al', 'Pizza', 'a', 'Detergente', 
            'Arroz', 'Naranja', 'Ambientador', '0%', 'Manzana', 'Agua', 
            '-', 'Zero', 'Atún', 'para', 'Zumo', 'Piel', 'Laca', 'Patatas',
            'Pavo', 'Labios', 'Uñas', 'Rubio', 'Caramelo', 'Recambio', 'Limpiador', 
           'congelada', 'Gluten', 'Talla', 'Original', 'del', 'Cola', 'Galletas', 
            'Cocido', 'D.O.']
numb = 100
words = []

words_list = [ i.strip().split(" ") for i in lista]

for i in words_list:
    for e in i:
        if e.replace(",", "").replace(".", "").isdigit() == False and e not in re_words:
            words.append(e)
            
word_counts = Counter(words)
    
most_common_words = word_counts.most_common(numb)

palabras = [i[0] for i in most_common_words]

In [99]:
#words_list

In [ ]:
'Leche', 'Vino', 'Natural', 'Gel', 'Colonia', 'Aceite', 'Kg', 'Pollo', 'Extra', 'Tinte', 'Sabor', 'Mini', 'Café', 'Tinto', 'Blanco', 'Azúcar', 'Galleta', 'Refresco', 'Jamón', 'Salsa',

In [28]:
f_df = df[df['unidades'].str.contains('Paquete')]

In [349]:
Paquete = i_df["unidades"].tolist()

In [24]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [25]:
texto_csv(df, "descripcion")

Column successfully exported to column_descripcion.csv
